# Notebook emulating plots from our WHAFT proposal

In [1]:
import xarray as xr
import gcsfs
from tqdm import tqdm
import fsspec
xr.set_options(display_style="html");

### list the netcdf files

In [2]:
fs = gcsfs.GCSFileSystem(project='ldeo-glaciology', mode='ab', cache_timeout = 0)
NCs = fs.glob('gs://ldeo-glaciology/AMPS/WRF_24/domain_03/wrf-20161225*')
print(f"Total of {len(NCs)} wrf files.")
print(NCs[0])

72
ldeo-glaciology/AMPS/WRF_24/domain_03/wrf-2016120100_f003-cf.nc


In [3]:
## load the first file to inialize the xarray
url = 'gs://' + NCs[0]
with  fsspec.open(url, mode='rb')  as openfile:  
    ds = xr.open_dataset(openfile, chunks={})  # these chunk sizes produce chunks of reasonable data volumes and which stretch through all time

## load the other files, each time concaternating them onto an xarray (AMPS) that grows in the time dimension each iteration. 
for i in tqdm(range(1, len(NCs)-1)):  
    url = 'gs://' + NCs[i]
    with  fsspec.open(url, mode='rb')  as openfile:  
        temp = xr.open_dataset(openfile, chunks={})  # these chunk sizes produce chunks of reasonable data volumes and which stretch through all time
    ds = xr.concat([AMPS,temp],'time')

100%|██████████| 70/70 [11:17<00:00,  9.68s/it]


I am interested to know why the cell above fills the memory, when my intention was to only be loading things lazily. Note that when I load a smaller number of files (e.g., 45), instead of the full 55, I produce an xarray which seems to be made up of dask arrays as intended, so I dont why does it take up so much space on disk? Is it something to do with the chunking? increasing chunk size with AMPS.chunk doesnt seems to change the memory usage (monitored with top). 

In [4]:
ds

<xarray.Dataset>
Dimensions:   (pressure: 5, south_north: 825, time: 71, west_east: 537)
Coordinates:
  * time      (time) datetime64[ns] 2016-12-01T03:00:00 ... 2016-12-09T21:00:00
  * pressure  (pressure) float32 1000.0 850.0 700.0 500.0 300.0
    lat       (south_north, west_east) float32 -80.58021 ... -67.988365
    lon       (south_north, west_east) float32 59.897705 59.80359 ... -157.98875
Dimensions without coordinates: south_north, west_east
Data variables:
    DateTime  (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    year      (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    month     (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    day       (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    hour      (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    minute    (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    Z_sfc     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    SST       (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    T_sfc     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    p_sfc     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    slp       (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    T_2m      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    Td_2m     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    r_v_2m    (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    q_2m      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    rh_2m     (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    u_10m_tr  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    v_10m_tr  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    ws_10m    (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    wd_10m    (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    precip_g  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    Z_p       (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    T_p       (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    Td_p      (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    r_v_p     (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    q_p       (time, pressure, south_north, west_east) float32 dask.array<chunksize=(1, 5, 825, 537), meta=np.ndarray>
    SW_d      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LW_d      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LW_u      (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LW_d_acc  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LW_u_acc  (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    albedo    (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    SH        (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    LH        (time, south_north, west_east) float32 dask.array<chunksize=(1, 825, 537), meta=np.ndarray>
    SH_acc    (time, south_north, west_east) float32 dask.array<chunksi

## Quick overview plot

In [ ]:
# plottime = 0

# if plottime == 0:
#     plottitle = 'hi'
#     plotdate = '23-Dec-2016 1800z'
# else:
#     plottitle = 'lo'
#     plotdate = '27-Dec-2016 1500z'
    

### Zoom in to Scott and Amundsen

same but take mean first...

### PLOT timeseries of 10m wind speeds 

In [ ]:
print(ds.lon.shape)
print(ds.lat.shape)
print(ds['ws_10m'].isel(time=0).shape)

### Get indices for Scott and Amundsen Glaciers

method 1 - mininmum distance search

method 2 - xarray labeled coordinates

In [ ]:
scott = (-154.2, -85.3)
amundsen = (-159.6, -85.4)

LAT = ds.lat.values[0]
LON = ds.lon.values[0]

a = abs( LAT - scott[1] ) + abs( LON - scott[0] )
si,sj = np.unravel_index(a.argmin(), a.shape)

a = abs( LAT - amundsen[1] ) + abs( LON - amundsen[0] )
ai,aj = np.unravel_index(a.argmin(), a.shape)

In [ ]:
# # rotate winds to earth-relative using the correct formulas
# ds['ue'] = ds['U10'] * ds['COSALPHA'] - ds['V10'] * ds['SINALPHA']
# ds['ve'] = ds['V10'] * ds['COSALPHA'] + ds['U10'] * ds['SINALPHA']

Crude method using indices

In [ ]:
print(ds['ws_10m'].isel(south_north=si, west_east=sj).argmax())
print(ds['time'].values[99])
print(ds['ws_10m'].isel(south_north=si, west_east=sj).argmin())
print(ds['time'].values[112])

plt.plot(ds['ws_10m'].isel(south_north=ai, west_east=aj), label='Amundsen')
plt.plot(ds['ws_10m'].isel(south_north=si, west_east=sj), label='Scott')
plt.ylabel('10 m Windspeed (m s$^{-1}$)')
plt.legend()
plt.savefig('figs/lineplot_WHAFT_ScottAmundsen_WS10_' + plottitle + '.pdf')

Better? method using labeled coordinates

### PLOT timeseries of other surface terms

### PLOT vertical profile of winds